In [1]:
import pandas as pd

In [69]:
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\archive\Student_Performance.csv")
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [71]:
df['Extracurricular Activities']=df['Extracurricular Activities'].astype('category').cat.codes
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,1,9,1,91.0
1,4,82,0,4,2,65.0
2,8,51,1,7,2,45.0
3,5,52,1,5,2,36.0
4,7,75,0,8,5,66.0


In [73]:
correlation_matrix = df.corr()
correlation_matrix

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
Hours Studied,1.000000,-0.012390,0.003873,0.001245,0.017463,0.373730
Previous Scores,-0.012390,1.000000,0.008369,0.005944,0.007888,0.915189
Extracurricular Activities,0.003873,0.008369,1.000000,-0.023284,0.013103,0.024525
Sleep Hours,0.001245,0.005944,-0.023284,1.000000,0.003990,0.048106
Sample Question Papers Practiced,0.017463,0.007888,0.013103,0.003990,1.000000,0.043268
Performance Index,0.373730,0.915189,0.024525,0.048106,0.043268,1.000000


In [67]:
import numpy as np
Y=df['Performance Index']
df.drop(columns=['Performance Index'],axis=0,inplace=True)
X=df.copy()


In [68]:
crr=Y.corr(X)
crr

ValueError: operands could not be broadcast together with shapes (10000,) (10000,5) 

In [5]:
X=np.array(X)
Y=np.array(Y)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,random_state=42,test_size=0.3)
xval,xtest,yval,ytest=train_test_split(X,Y,random_state=42,test_size=0.5)


In [6]:
Y.shape

(10000,)

In [50]:
from torch.utils.data import Dataset,DataLoader
import torch
class Customdataset(Dataset):
    def __init__(self,a,b):
        self.x=torch.tensor(a,dtype=torch.float32)
        self.y=torch.tensor(b,dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
traindataset=Customdataset(a=xtrain,b=ytrain)
valdataset=Customdataset(a=xval,b=yval)
testdataset=Customdataset(a=xtest,b=ytest)

traindl=DataLoader(traindataset,batch_size=10,shuffle=True)
testdl=DataLoader(testdataset,batch_size=10,shuffle=True)
valdl=DataLoader(valdataset,batch_size=10,shuffle=True)

In [60]:
import torch.nn as nn

class regression(nn.Module):
    def __init__(self):
        super(regression,self).__init__()
        self.Linear1=nn.Linear(X.shape[1],20)
        self.Linear2=nn.Linear(20,1)

    def forward(self,x):
        x=self.Linear1(x)
        x=self.Linear2(x)
        return x

model=regression()

In [61]:
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=1e-3)
loss=nn.MSELoss()

In [63]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

for epoch in range(10):
    model.train()
    ttruth,ttrial=[],[]
    for values,lables in traindl:
        optimizer.zero_grad()
        output=model(values)
        bloss=loss(output,lables).squeeze()
        ttruth.append((lables.detach().numpy()).flatten())
        ttrial.append((output.detach().numpy()).flatten())
        bloss.backward()
        optimizer.step()
        optimizer.zero_grad()
    vtruth,vtrial=[],[]
    model.eval()
    with torch.no_grad():
        for values,lables in valdl:
            output=model(values).squeeze()
            vtruth.append((lables.detach().numpy()).flatten())
            vtrial.append((output.detach().numpy()).flatten())
    print(f'''
    train:
          MSE:{mean_squared_error(ttruth,ttrial)},MAE:{mean_absolute_error(ttruth,ttrial)},R2:{r2_score(ttruth,ttrial)}
    ''')
    print(f'''
    val:
          MSE:{mean_squared_error(vtruth,vtrial)},MAE:{mean_absolute_error(vtruth,vtrial)},R2:{r2_score(vtruth,vtrial)}
    ''')


c:\Users\ASUS\.conda\envs\PyTorch\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



    train:
          MSE:116.26485639943255,MAE:8.94180691419329,R2:0.6835622354961416
    

    val:
          MSE:139.38115512093898,MAE:9.881051323318482,R2:0.6196484346929914
    


c:\Users\ASUS\.conda\envs\PyTorch\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



    train:
          MSE:143.98312639192582,MAE:10.053192298344204,R2:0.6088508864680581
    

    val:
          MSE:164.18153990095468,MAE:10.722393339920043,R2:0.5515170779387426
    


c:\Users\ASUS\.conda\envs\PyTorch\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...


    train:
          MSE:192.85040107906931,MAE:11.63987928227016,R2:0.47609523184156155
    

    val:
          MSE:205.20686213225162,MAE:12.013291976165771,R2:0.440469755104209
    


c:\Users\ASUS\.conda\envs\PyTorch\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



    train:
          MSE:198.30401991394848,MAE:11.802153497423447,R2:0.4610067093526381
    

    val:
          MSE:187.5257439345615,MAE:11.498472415924073,R2:0.48851506627576563
    


c:\Users\ASUS\.conda\envs\PyTorch\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



    train:
          MSE:194.75680164384326,MAE:11.677412817273822,R2:0.469942813148037
    

    val:
          MSE:190.36724599493098,MAE:11.594106058502197,R2:0.48042519163970787
    


In [74]:
vtruth,vtrial=[],[]
with torch.no_grad():
    for values,lables in testdl:
            output=model(values).squeeze()
            vtruth.append((lables.detach().numpy()).flatten())
            vtrial.append((output.detach().numpy()).flatten())
    print(f'''
    val:
          MSE:{mean_squared_error(vtruth,vtrial)},MAE:{mean_absolute_error(vtruth,vtrial)},R2:{r2_score(vtruth,vtrial)}
    ''')


    val:
          MSE:192.68932338939004,MAE:11.637257965087892,R2:0.4789810034302599
    


In [75]:
model_state = model.state_dict()
torch.save(model_state, 'model_saved_weights.pt')